In [16]:
import pandas as pd
import sqlite3
import re
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#incarcam datele
results = pd.read_csv("results.csv")
shootouts = pd.read_csv("shootouts.csv")

In [3]:
def connect_to_database(database_name):
    conn = sqlite3.connect(database_name)
    cursor = conn.cursor()
    return conn, cursor

# Conectare la baza de date
conn, cursor = connect_to_database('football.db')

results.to_sql('results',conn, if_exists='replace', index=False)
shootouts.to_sql('shootouts', conn, if_exists='replace', index=False)

# Adăugare coloane 'day', 'month' și 'year' în tabela 'results', dacă nu există deja
cursor.execute("""
    PRAGMA table_info(results);
""")
columns = cursor.fetchall()
column_names = [column[1] for column in columns]
if 'day' not in column_names:
    cursor.execute("""
        ALTER TABLE results
        ADD COLUMN day INTEGER;
    """)
if 'month' not in column_names:
    cursor.execute("""
        ALTER TABLE results
        ADD COLUMN month INTEGER;
    """)
if 'year' not in column_names:
    cursor.execute("""
        ALTER TABLE results
        ADD COLUMN year INTEGER;
    """)

# Adăugare coloane 'day', 'month' și 'year' în tabela 'shootouts', dacă nu există deja
cursor.execute("""
    PRAGMA table_info(shootouts);
""")
columns = cursor.fetchall()
column_names = [column[1] for column in columns]
if 'day' not in column_names:
    cursor.execute("""
        ALTER TABLE shootouts
        ADD COLUMN day INTEGER;
    """)
if 'month' not in column_names:
    cursor.execute("""
        ALTER TABLE shootouts
        ADD COLUMN month INTEGER;
    """)
if 'year' not in column_names:
    cursor.execute("""
        ALTER TABLE shootouts
        ADD COLUMN year INTEGER;
    """)


# Salvează modificările în baza de date
conn.commit()

#inchidem conexiunea la baza de date
conn.close()

In [4]:
def extract_date_info(date_str):
    try:
        month, day, year = map(int, date_str.split('/'))
        if year >= 1900:
            return day, month, year
    except ValueError:
        pass

    match = re.match(r'(\d{1,2})-(\d{1,2})-(\d{4})', date_str)
    if match:
        groups = match.groups()
        month, day, year = map(int, (groups[0], groups[1], groups[2]))
        return day, month, year

    match = re.match(r'(\d{4})-(\d{2})-(\d{2})', date_str)
    if match:
        groups = match.groups()
        year, month, day = map(int, (groups[0], groups[1], groups[2]))
        return day, month, year

    return None, None, None

# Conectare la baza de date
conn = sqlite3.connect('football.db')
cursor = conn.cursor()

# Actualizează valorile pentru coloanele 'day', 'month', 'year' folosind datele extrase
cursor.execute("SELECT date FROM results;")
dates = cursor.fetchall()

for date_row in dates:
    date_str = date_row[0]
    day, month, year = extract_date_info(date_str)
    cursor.execute("UPDATE results SET day=?, month=?, year=? WHERE date=?", (day, month, year, date_str))

# Actualizează valorile pentru coloanele 'day', 'month', 'year' pentru tabela 'shootouts'
cursor.execute("SELECT date FROM shootouts;")
dates_shootouts = cursor.fetchall()

for date_row in dates_shootouts:
    date_str = date_row[0]
    day, month, year = extract_date_info(date_str)
    cursor.execute("UPDATE shootouts SET day=?, month=?, year=? WHERE date=?", (day, month, year, date_str))

cursor.execute("ALTER TABLE results DROP COLUMN date")
cursor.execute("ALTER TABLE shootouts DROP COLUMN date")

# Actualizare coloana 'first_shooter' cu valoarea 'Necunoscut' pentru celulele 'null'
cursor.execute("UPDATE shootouts SET first_shooter='Necunoscut' WHERE first_shooter IS NULL")

conn.commit() # Salvează modificarile în baza de date

# Închide conexiunea la baza de date
conn.close()

In [36]:
conn = sqlite3.connect('football.db')

df_results = pd.read_sql('SELECT * FROM results', conn)
df_shootouts = pd.read_sql('SELECT * FROM shootouts', conn)

def assign_unique_codes(df, column, start_code):
    unique_codes = {}
    code = start_code
    for value in df[column].unique():
        if pd.notnull(value):  # Ignorăm valorile NaN
            unique_codes[value] = code
            code += 1  # Incrementăm codul pentru următoarea valoare unică
    return unique_codes, code # Returnam si ultimul cod utilizat

# Atribuim coduri unice pentru coloana 'home_team'
unique_codes_home_team, code = assign_unique_codes(df_results, 'home_team', 100)

# Aplicăm aceleași coduri unice pentru coloanele 'away_team' și 'country'
df_results['home_team_code'] = df_results['home_team'].map(unique_codes_home_team)
df_results['away_team_code'] = df_results['away_team'].map(unique_codes_home_team)
df_results['country_code'] = df_results['country'].map(unique_codes_home_team)

# Remediem valorile de tip float din 'away_team_code' și 'country_code' folosind fillna()
df_results['away_team_code'] = df_results['away_team_code'].fillna(df_results['home_team_code'])
df_results['country_code'] = df_results['country_code'].fillna(df_results['home_team_code'])

# Convertim valorile din coloanele 'away_team_code' și 'country_code' la întreg
df_results['away_team_code'] = df_results['away_team_code'].astype(int)
df_results['country_code'] = df_results['country_code'].astype(int)


# Atribuim coduri unice pentru coloana 'tournament', pornind de la valoarea maximă existentă
unique_codes_tournament, code = assign_unique_codes(df_results, 'tournament', code)
df_results['tournament_code'] = df_results['tournament'].map(unique_codes_tournament)

# Inițializăm coloana 'city_code' cu valori nule
df_results['city_code'] = np.nan

# Atribuim coduri unice pentru coloana 'city', pornind de la valoarea maximă existentă
unique_codes_city, _ = assign_unique_codes(df_results, 'city', code)
df_results['city_code'] = df_results['city'].map(unique_codes_city)

# Pentru tabela shootouts
# Atribuirea codurilor pentru coloanele home_team, away_team, winner
df_shootouts['home_team_code'] = df_shootouts['home_team'].map(unique_codes_home_team)
df_shootouts['away_team_code'] = df_shootouts['away_team'].map(unique_codes_home_team)
df_shootouts['winner_code'] = df_shootouts['winner'].map(unique_codes_home_team)

_, code = assign_unique_codes(df_shootouts, 'first_shooter', code)
unique_codes_first_shooter, _ = assign_unique_codes(df_shootouts, 'first_shooter', code)
df_shootouts['first_shooter_code'] = df_shootouts['first_shooter'].map(unique_codes_first_shooter)

# Afisăm primele câteva rânduri din DataFrame-ul modificat
print(df_results.head())
print(df_shootouts.head())

conn.close()


  home_team away_team  home_score  away_score tournament     city   country  \
0  Scotland   England           0           0   Friendly  Glasgow  Scotland   
1   England  Scotland           4           2   Friendly   London   England   
2  Scotland   England           2           1   Friendly  Glasgow  Scotland   
3   England  Scotland           2           2   Friendly   London   England   
4  Scotland   England           3           0   Friendly  Glasgow  Scotland   

   neutral  day  month  year  home_team_code  away_team_code  country_code  \
0        0   30     11  1872             100             101           100   
1        0    8      3  1873             101             100           101   
2        0    7      3  1874             100             101           100   
3        0    6      3  1875             101             100           101   
4        0    4      3  1876             100             101           100   

   tournament_code  city_code  
0              427      